In [ ]:
DESC TABLE CUSTOMER_REVIEWS;

In [ ]:
DESC TABLE SHIPPING_LOGS;

In [ ]:
CREATE OR REPLACE TABLE merged_reviews AS
SELECT 
  r.order_id,
  r.filename,
  r.product,
  r.review_date,
  r.review_text,
  s.shipping_date,
  s.carrier,
  s.tracking_number,
  s.latitude,
  s.longitude,
  s.status,
  s.delivery_days,
  s.late,
  s.region
FROM 
  customer_reviews r
JOIN 
  shipping_logs s
ON 
  r.order_id = s.order_id;

In [ ]:
from snowflake.snowpark.context import get_active_session

session = get_active_session()
# Load parsed reviews into a DataFrame
snowpark_df = session.sql("SELECT * FROM merged_reviews")

# Show a sample of the data
snowpark_df.show()

In [ ]:
from snowflake.snowpark.functions import lower, trim, col

# Standardize the review text
df_lowercase = snowpark_df.with_column("review_text", trim(lower(col("review_text"))))

In [ ]:
df_lowercase.write.mode("overwrite").save_as_table("clean_reviews")